# JS08 - Stream Workers

## Overview

Macrometa GDN allows you to integrate streaming data and take appropriate actions. Most stream processing use cases involve collecting, analyzing, and integrating or acting on data generated during business activities by various sources.

| Stage | Description | 
|  :----:  |    :----:   |
| Collect | Receive or capture data from various data sources.| 
| Analyze | Analyze data to identify interesting patterns and extract information.| 
| Act | Take actions based on the findings. For example, running simple code, calling an external service, or triggering a complex integration.| 
| Integrate | Provide processed data for consumer consumption.| 

You can process streams to perform the following actions with your data:

- Transform data from one format to another. For example, from XML to JSON.
- Enrich data received from a specific source by combining it with databases and services.
- Correlate data by joining multiple streams to create an aggregate stream.
- Clean data by filtering it and by modifying the content in messages. For example, obfuscating sensitive information.
- Derive insights by identifying event patterns in data streams.
- Summarize data with time windows and incremental aggregations.
- Real-time ETL for collections, tailing files, and scraping HTTP endpoints.
- Integrating stream data and trigger actions based on the data. This can be a single service request or a complex enterprise integration flow.

In this tutorial we will build a simple stream worker for finding various heart rate measures like average bpm, minimum bpm etc.

## Pre-requisite

Lets Assume 
- you have already made a tenant account, and have a username and password
- you have installed the jsc8 drivers as explained in section 01
- you have generated an API Key as explained in section 01


In [ ]:
#/* run this once to install javascript kernal and jsc8 in google colab, then reload, and then skip this
!npm install jsc8
!npm install -g --unsafe-perm ijavascript
!ijsinstall --install=global  # */

## 1. Importing Libraries & Define Variables

In [ ]:
// Variables - Queries
const global_url = "https://gdn.paas.macrometa.io";
const token = ""; // <-- Token goes here if applicable
const apiKey = ""; // <-- apiKey goes here if applicable
const email = "email"; // <-- Email goes here
const password = "password"; // <-- password goes here
const geo_fabric = "_system";
const heart_rates_collection = "HeartRates";
const heart_rates_statistics_collection = "HeartRateStatistics";

const heart_rate_statistics_worker = "HeartRateStatisticsWorker";
const mock_heart_rate_data_generator = "MockHeartRateDataGenerator";

## 2. Connecting to GDN

In [ ]:
const jsc8 = require("jsc8");

// Simple Way
// const client = new jsc8({
//   url: global_url,
//   token: token,
//   fabricName: geo_fabric,
// });

// ----- OR -----
// const client = new jsc8({url: global_url, apiKey: apiKey, fabricName: geo_fabric});

// To use advanced options
const client = new jsc8(global_url);
client.useFabric(geo_fabric);
client
  .login(email, password)
  .then((result) => console.log("Login successfully", result))
  .catch((err) => console.error("Error while login", err.message));

## 3. Creating Collections

In [ ]:
async function createCollection() {
  console.log("\n 3. CREATE_COLLECTION");

  try {
    console.log(
      `Creating the HEART_RATES_COLLECTION ${heart_rates_collection}...`
    );
    let isCollectionExists = await client.hasCollection(heart_rates_collection);
    if (!isCollectionExists) {
      await client.createCollection(heart_rates_collection, { stream: true });
    }

    console.log(
      `Creating the HEART_RATES_STATISTICS_COLLECTION ${heart_rates_statistics_collection}...`
    );
    isCollectionExists = await client.hasCollection(
      heart_rates_statistics_collection
    );
    if (!isCollectionExists) {
      await client.createCollection(heart_rates_statistics_collection);
    }
  } catch (e) {
    console.log("Collection creation did not succeed due to " + e.message);
  }
}

createCollection();


## 4. Validate Stream Application

### 4.1 Validating heart rate simulator definition

#### Option1: Use mockaroo api to generate the mock heart rate data 

NOTE: If you are using mockaroo API then you need to signup to mockaroo and past the API Key. You can find API key here https://www.mockaroo.com/myaccount

In [ ]:
const mockarooAPIKey = "XXXX";
const dataGeneratorAppDefinition = `
@App:name('MockHeartRateDataGenerator')
@App:description('Mock data generator by calling mockaroo api for heart rate')
@App:qlVersion('2')


CREATE TRIGGER HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );

CREATE TABLE HeartRates (name string, bpm int);

CREATE SINK MockarooServiceCallSink WITH (type='http-call', sink.id='mockaroo-service', publisher.url='https://api.mockaroo.com/api/a6e130b0?count=10&key=${mockarooAPIKey}', map.type='json', method='GET') (triggered_time string);

CREATE SOURCE MockarooServiceResponseSink WITH (type='http-call-response', sink.id='mockaroo-service', map.type='json', http.status.code='200') (name string, bpm int);


INSERT INTO MockarooServiceCallSink
SELECT time:currentTimestamp() as triggered_time 
FROM HeartRateDataGeneratorTrigger;

-- Note: Consume data received from the external service
@info(name = 'ConsumeProcessedData')
INSERT INTO HeartRates
SELECT name, bpm
FROM MockarooServiceResponseSink;
`;

#### Option2: Use custom stream worker to generate the heart rate data 

In [ ]:
const dataGeneratorAppDefinition = `
@App:name("MockHeartRateDataGenerator")
@App:qlVersion("2")

CREATE TRIGGER HeartRateDataGeneratorTrigger WITH ( interval = 10 sec );

CREATE TABLE HeartRates (name string, bpm int);


-- Note: Generating random bpm and name 
@info(name = 'ConsumeProcessedData')
INSERT INTO HeartRates
SELECT 
js:eval("['Vasili', 'Rivalee', 'Betty', 'Jennifer', 'Alane', 'Sarena', 'Bruno', 'Carolee', 'Emmott', 'Andre'][Math.floor(Math.random() * 10)]","string") as name,
js:eval("Math.floor(Math.random() * 40) + 40","int") as bpm
FROM HeartRateDataGeneratorTrigger;
`

### 4.2 Validating stream worker

In [ ]:
const statisticAppDefinition = `
@App:name('HeartRateStatisticsWorker')
@App:description("Calculate the statistics for heart rates")
@App:qlVersion("2")

CREATE SOURCE HeartRates WITH (type = 'database', collection = "HeartRates", collection.type="doc", replication.type="global", map.type='json') (name string, bpm int);

CREATE TABLE HeartRateStatistics (eventTime long, name string, minBpm int, maxBpm int, avgBpm double);


INSERT INTO HeartRateStatistics
SELECT 
    eventTimestamp() as eventTime,
    name as name,
    min(bpm) as minBpm,
    max(bpm) as maxBpm,
    avg(bpm) as avgBpm
FROM HeartRates window sliding_time(1 min)
group by name
`;

async function validateStreamApp() {
  console.log(`\n 4. VALIDATE_STREAM_WORKERS ... region ${global_url}`);

  console.log("--- Validating Stream Application Definition");
  let result = await client.validateStreamApp(statisticAppDefinition);
  console.log(
    `--- Validated Stream Application Definition ${heart_rate_statistics_worker} ---`,
    result.message
  );
  result = await client.validateStreamApp(dataGeneratorAppDefinition);
  console.log(
    `--- Validated Stream Application Definition ${mock_heart_rate_data_generator} ---`,
    result.message
  );
}

validateStreamApp().catch((err) =>
  console.log("Error while validating streamApp :", err.message)
);

## 5. Save Stream Application

In [ ]:
async function createStreamApp() {
  console.log(`\n 5. CREATE_STREAM_WORKERS ... region ${global_url}`);

  let result;

  console.log("--- Creating Stream Application");
  let streamAppList = await client.retrieveStreamApp();
  streamAppList = streamAppList.streamApps.map((streamApp) => streamApp.name);
  // The stream app will be created by default in the local region. Optionally, you can send dclist to deploy stream
  if (!streamAppList.includes(heart_rate_statistics_worker)) {
    result = await client.createStreamApp([], statisticAppDefinition);
    console.log(
      `--- Created Stream Application ${heart_rate_statistics_worker} ---`
    );
  }

  if (!streamAppList.includes(mock_heart_rate_data_generator)) {
    result = await client.createStreamApp([], dataGeneratorAppDefinition);
    console.log(
      `--- Created Stream Application ${mock_heart_rate_data_generator} ---`
    );
  }
}

createStreamApp().catch((err) =>
  console.log("Error while creating streamApp :", err)
);


## 6. Publish Stream Application

In [ ]:
async function activateStreamApp() {
  console.log(`\n 6. Activating Stream Workers`);
  await client.activateStreamApp(heart_rate_statistics_worker, true);
  await client.activateStreamApp(mock_heart_rate_data_generator, true);
  console.log(`6. Activated Stream Workers`);
}

activateStreamApp().catch((err) =>
  console.log("Error while publishing streamApp :", err)
);

## 7. Checking HeartRates documents using C8QL

In [ ]:
// Please wait for 1 minute after executing above step as we used sliding window of 1 minute
async function c8Queries() {
  console.log(`\n 7. LIST_HEART_RATE_DATA in region ${global_url}`);
  let docs = await client.executeQuery(
    `FOR doc IN ${heart_rates_collection} LIMIT 0, 5 RETURN doc`
  );
  console.log(docs);
}

c8Queries();

## 8. Checking HeartRateStatistics documents using C8QL

In [ ]:
async function c8Queries() {
  console.log(`\n 8. LIST_HEART_RATE_STATISTICS_DATA in region ${global_url}`);
  let docs = await client.executeQuery(
    `FOR doc IN ${heart_rates_statistics_collection} RETURN doc`
  );
  console.log(docs);
}

c8Queries();

## 9.  Delete StreamApp and Collections

In [ ]:
async function deleteData() {
  console.log("\n 9. DELETE_DATA");
  await client.deleteStreamApp(heart_rate_statistics_worker);
  await client.deleteStreamApp(mock_heart_rate_data_generator);
  await client.deleteCollection(heart_rates_collection);
  await client.deleteCollection(heart_rates_statistics_collection);
  console.log("StreamApp and Collection deleted");
}

deleteData().catch((err) =>
  console.log("Error while deleting streamApp and collection:", err.message)
);

## Section Completed!

TBC...